In [111]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.datasets import mnist

In [171]:
#Define Strategy
strategy = tf.distribute.MirroredStrategy()
print("Number of devices:{}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices:1


In [172]:
#Set batch size accoring to number of replicas
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
BUFFER_SIZE = 10000

In [173]:
#Define Train and test data
(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_y = tf.keras.utils.to_categorical(train_y, 10)
test_y = tf.keras.utils.to_categorical(test_y, 10)

In [174]:
#Scale function
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image = image/255.0
  image = tf.expand_dims(image, axis=-1)
  return image, label
#Dataset for training
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.map(scale)
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
#Dataset for testing
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_ds = test_ds.map(scale)
test_ds = test_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [175]:
#Model with strategy
with strategy.scope():
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
                                      tf.keras.layers.MaxPool2D((2,2)),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(10, activation='softmax')
  ])

In [176]:
#Fit model
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
938/938 [==============================] - 12s 9ms/step - loss: 0.4697 - accuracy: 0.8741 - val_loss: 0.1200 - val_accuracy: 0.9653
Epoch 2/5
938/938 [==============================] - 8s 8ms/step - loss: 0.1118 - accuracy: 0.9675 - val_loss: 0.0781 - val_accuracy: 0.9768
Epoch 3/5
938/938 [==============================] - 8s 8ms/step - loss: 0.0762 - accuracy: 0.9780 - val_loss: 0.0692 - val_accuracy: 0.9782
Epoch 4/5
938/938 [==============================] - 8s 8ms/step - loss: 0.0608 - accuracy: 0.9823 - val_loss: 0.0597 - val_accuracy: 0.9811
Epoch 5/5
938/938 [==============================] - 8s 8ms/step - loss: 0.0555 - accuracy: 0.9837 - val_loss: 0.0593 - val_accuracy: 0.9815
